## Building flask api

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')

In [4]:
from src.serve import *

In [5]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [11]:
data_dir = 'data/midi/v12/'

file_path = Path('../../')
path = file_path/data_dir/'midi_encode'
out_path = file_path/'data/generated/'

In [12]:
encoded_path = file_path/'data/assets/cmajor/seed'

## Learn config

In [13]:
import hashlib
import shutil

In [16]:
def df2records(path):
    df = pd.read_csv(path/'midi_encode.csv')
    df = df.loc[df['numpy'].notna()] # make sure it exists
    df = df.loc[df.source == 'hooktheory_c'] # hooktheory only
    df = df.reindex(index=df.index[::-1]) # A's first
    df = df.where((pd.notnull(df)), None) # nan values break json
    return df.to_dict('records')

In [17]:
df2records(path)

[{'ht_bpm': 128.0,
  'md5': '67a12c729c20a3bc49e22e1da46d472e',
  'numpy': 'midi_encode/hooktheory/pianoroll/a/archie/famous/chorus_key_cmajor.npy',
  'ht_time_signature': 4.0,
  'midi_title': 'Famous',
  'source': 'hooktheory_c',
  'artist': 'archie',
  'ht_offset': 0.0,
  'section': 'chorus',
  'song_url': 'https://www.hooktheory.com/theorytab/view/archie/famous',
  'mxl': None,
  'ht_key': 'C',
  'parts': 'chorus',
  'title': 'famous',
  'genres': 'Electronic',
  'midi': 'midi_sources/hooktheory/pianoroll/a/archie/famous/chorus_key_cmajor.mid',
  'ht_mode': 1.0},
 {'ht_bpm': 70.0,
  'md5': '49e4fef90cc080d43618180d4a32d0ff',
  'numpy': 'midi_encode/hooktheory/pianoroll/a/apink/promise-u/chorus_key_cmajor.npy',
  'ht_time_signature': 4.0,
  'midi_title': 'pu',
  'source': 'hooktheory_c',
  'artist': 'apink',
  'ht_offset': 0.0,
  'section': 'chorus',
  'song_url': 'https://www.hooktheory.com/theorytab/view/apink/promise-u',
  'mxl': None,
  'ht_key': 'C',
  'parts': 'chorus',
  'titl

In [21]:
def format_meta(s):
    title = s['title'].title().replace('-', ' ')
    artist = s['artist'].title().replace('-', ' ')
    display = ' - '.join([title, artist])
    if s.get('section'): display += ' - ' + s['section'].title()
    sid = s['md5']
    
    json_data = {
        'title': title,
        'artist': artist,
        'bpm': s['ht_bpm'],
        'display': display,
        'genres': s['genres'],
        'sid': sid
    }
    source_file = file_path/data_dir/s['midi']
#     key_file = str(source_file).replace('cmajor', 'original')
    key_file = source_file
    to_file = encoded_path/f'{sid[::-1]}.mid'
    to_json = encoded_path/f'{sid[::-1]}.json'
    if not to_file.exists():
        to_file.mkdir(parents=True, exist_ok=True)
        shutil.copy(str(key_file), str(to_file))
        with open(to_json, 'w') as jp:
            json.dump(json_data, jp)
    
    return json_data

In [19]:
def build_db(path):
    recordlist = df2records(path)
    htlist = [format_meta(s) for s in recordlist]
    json_path = file_path/'data/assets/cmajor/json/htlist.json'
    json_path.mkdir(parents=True, exist_ok=True)
    with open(json_path, 'w') as fp:
        json.dump(htlist, fp, separators=(',', ':'))
    return htlist

In [22]:
htlist = build_db(path)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/assets/cmajor/json/htlist.json'